#  Graph RAG using LangChain and Kuzu 

Taken  from https://pypi.org/project/langchain-kuzu/ 

In [1]:
# A default setup cell.
# It imports environment variables, define 'devtools.debug" as a buildins, set PYTHONPATH, and code auto-reload
# Copy it in other Notebooks


from dotenv import load_dotenv
from rich import print

load_dotenv(verbose=True)
%load_ext autoreload
%autoreload 2
%reset -f

# cSpell: disable

In [2]:
import kuzu
from ipycytoscape import CytoscapeWidget
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_kuzu.chains.graph_qa.kuzu import KuzuQAChain
from langchain_kuzu.graphs.kuzu_graph import KuzuGraph
from langchain_openai import ChatOpenAI

from src.ai_core.llm import get_llm

2025-04-02 15:55:50.334 | INFO     | src.utils.config_mngr:singleton:99 - selected config=training_edenai
2025-04-02 15:55:50.345 | INFO     | src.ai_core.cache:set_method:89 - LLM cache : InMemoryCache


In [3]:
DB = "test_db4"
MODEL_ID = None
llm = get_llm(llm_id=MODEL_ID)

# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

text = "Tim Cook is the CEO of Apple. Apple has its headquarters in California."

if True:  # ne schema
    allowed_nodes = ["Person", "Company", "Location"]
    allowed_relationships = [
        ("Person", "IS_CEO_OF", "Company"),
        ("Company", "HAS_HEADQUARTERS_IN", "Location"),
    ]
    # Define the LLMGraphTransformer
    llm_transformer = LLMGraphTransformer(
        llm=llm,
        allowed_nodes=allowed_nodes,
        allowed_relationships=allowed_relationships,
    )

    # Convert the given text into graph documents
    documents = [Document(page_content=text)]
    graph_documents = llm_transformer.convert_to_graph_documents(documents)
    db = kuzu.Database(DB)
    graph = KuzuGraph(db, allow_dangerous_requests=True)

    # Add the graph document to the graph
    graph.add_graph_documents(
        graph_documents,
        include_source=True,
    )
else:
    print("load {DB}")

2025-04-02 15:56:25.556 | INFO     | src.ai_core.llm:get_llm:497 - get LLM:'gpt_4omini_edenai'


In [4]:
print(llm_transformer)

In [5]:
llm = get_llm(llm_id="gpt_4o_edenai")

2025-04-02 15:56:56.883 | INFO     | src.ai_core.llm:get_llm:497 - get LLM:'gpt_4o_edenai'


In [13]:
db = kuzu.Database(DB)
graph = KuzuGraph(db, allow_dangerous_requests=True)

# Create the KuzuQAChain with verbosity enabled to see the generated Cypher queries
chain = KuzuQAChain.from_llm(
    llm=get_llm(llm_id="gpt_4o_edenai"),
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)

# Query the graph
queries = [
    "Who is the CEO of Apple?",
    "Where is Apple headquartered?",
]

for query in queries:
    result = chain.invoke(query)
    print(f"Query: {query}\nResult: {result}\n")

2025-04-02 16:21:14.526 | INFO     | src.ai_core.llm:get_llm:497 - get LLM:'gpt_4o_edenai'




> Entering new KuzuQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:IS_CEO_OF]->(c:Company) WHERE LOWER(c.id) = LOWER('Apple') RETURN p.id
Full Context:
[{'p.id': 'Tim Cook'}]

> Finished chain.
Query: Who is the CEO of Apple?
Result: {'query': 'Who is the CEO of Apple?', 'result': 'Tim Cook is the CEO of Apple.'}



> Entering new KuzuQAChain chain...
Generated Cypher:
MATCH (c:Company)-[:HAS_HEADQUARTERS_IN]->(l:Location) WHERE LOWER(c.id) = LOWER('Apple') RETURN l.id
Full Context:
[{'l.id': 'California'}]

> Finished chain.
Query: Where is Apple headquartered?
Result: {'query': 'Where is Apple headquartered?', 'result': "I don't know the answer."}



In [9]:
# Create Cytoscape widget
from src.webapp.ui_components.cypher_graph_display import get_cytoscape_json, get_cytoscape_style

cyto = CytoscapeWidget()
cyto.graph.add_graph_from_json(get_cytoscape_json(graph))
cyto.set_style(get_cytoscape_style())
# Set layout and style
cyto.set_layout(animate=True)

In [10]:
# Display the graph
cyto

CytoscapeWidget(cytoscape_layout={'name': 'cola', 'animate': True}, cytoscape_style=[{'selector': 'node', 'css…